In [ ]:
from PIL import Image
from surya.recognition import RecognitionPredictor
from surya.detection import DetectionPredictor

In [ ]:
image = Image.open("sample_ocr_image.png")
recognition_predictor = RecognitionPredictor()
detection_predictor = DetectionPredictor()

In [ ]:
predictions = recognition_predictor([image], det_predictor=detection_predictor)

In [ ]:
from surya.layout import LayoutPredictor

image = Image.open("sample_ocr_image.png")
layout_predictor = LayoutPredictor()

# layout_predictions is a list of dicts, one per image
layout_predictions = layout_predictor([image])


In [ ]:
layout_predictions

In [ ]:
from PIL import Image
from surya.recognition import RecognitionPredictor
from surya.detection import DetectionPredictor
from surya.layout import LayoutPredictor

# Initialize models
layout_predictor = LayoutPredictor()
detection_predictor = DetectionPredictor()
recognition_predictor = RecognitionPredictor()

# Load image
image = Image.open("sample_ocr_image.png").convert("RGB")

# 1. Layout analysis
layout_predictions = layout_predictor([image])
layout_boxes = layout_predictions[0].bboxes  # List[LayoutBox]

# Sort by reading order
layout_boxes = sorted(layout_boxes, key=lambda x: x.position)

full_text_blocks = []

# 2. Process each layout block
for layout_box in layout_boxes:
    x1, y1, x2, y2 = layout_box.bbox
    cropped_image = image.crop((x1, y1, x2, y2))

    # Recognize text inside cropped region
    predictions = recognition_predictor([cropped_image], det_predictor=detection_predictor)

    if not predictions or len(predictions[0].text_lines) == 0:
        continue  # Skip if no text detected

    text_lines = predictions[0].text_lines
    block_text = "\n".join([line.text for line in text_lines])

    if block_text.strip():
        full_text_blocks.append(block_text)

# Combine everything into final text
final_text = "\n\n".join(full_text_blocks)

print("=== FULL OCR TEXT ===")
print(final_text)


 ## TROCR

In [7]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests
import cv2
from PIL import Image

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
#image = cv2.imread('sample_ocr_image.png')
url = "https://jeroen.github.io/images/testocr.png"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")


pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(generated_text)

ITEMOUNT
